In [1]:
import numpy as np
import cv2
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
cap = cv2.VideoCapture('./train/trainGray.m4v')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bufTemp = np.empty((frameCount, frameHeight * frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, x = np.array(cap.read())
    x = x[:,:,0]
    x.shape = frameHeight*frameWidth
    bufTemp[fc] = x
    fc += 1

cap.release()

# x = bufTemp[0, 0:frameHeight*frameWidth]
# x.shape = (frameHeight, frameWidth)

In [3]:
x = np.empty((frameCount-1, (frameHeight*frameWidth)+1), np.dtype('float32'))
print(x.shape)
print(x[0].shape)
x[0,0] = 1
print(type(x[0,0]))
print(x[0,0])
for j in range(1,frameCount):
    i = j-1
    firstFrame = np.array(bufTemp[i], dtype='int16')
    secondFrame = np.array(bufTemp[j], dtype='int16')
    x[i,0] = 1
    x[i,1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
    x[i,:] = np.absolute(x[i,:])
    mean = np.mean(x[i,:])
    x[i,:] = x[i,:] - mean
    x[i,:] = x[i,:]/np.std(x[i,:])

(20399, 73601)
(73601,)
<class 'numpy.float32'>
1.0


In [4]:
bufTemp = 0
y = np.loadtxt('./train/train.txt', np.dtype('float64'))

m = frameCount-1
trainM = int(m*.6)
valM = int((m - trainM)*.5)
testM = m-(trainM + valM)

rng_state = np.random.get_state()
np.random.shuffle(x)
np.random.set_state(rng_state)
np.random.shuffle(y)
# np.random.shuffle(x)

trainX = x[0:trainM]
valX = x[trainM:trainM+valM]
testX = x[trainM+valM:trainM+valM+testM]

trainY = y[1:trainM+1]
valY = y[trainM+1:trainM+valM+1]
testY = y[trainM+valM+1:trainM+valM+testM+1]
# y = 0
print(m, trainM, valM, testM)
print(len(trainX),len(trainY),len(valX),len(valY),len(testX),len(testY))

# Alpha greater than 0.16 will overflow and not work.
# Alpha around 0.01 or lower seems to do the best.
# alpha = np.array([0.1],np.dtype('float64'))
alpha2 = np.array([0.00000610351563,0.0000122070313,0.0000244140625,0.000048828125,0.00009765625,0.0001953125,0.000390625,0.00078125,0.0015625,0.003125,0.00625,0.0125,0.025,0.05,0.1],np.dtype('float64'))
alpha = np.array([0.0001953125],np.dtype('float64'))
lmbda = np.array([0.64],np.dtype('float64'))

outerLoopCount = len(alpha2)
loopCount = 20 # Was 5
costOnLoop = 20

difDivider = 2


alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))

alpLmbCost = np.zeros((len(alpha), len(lmbda)), np.dtype('float64'))
costArr = np.zeros((1),np.dtype('float64'))
beginArr = 1

result = np.empty((len(alpha), len(lmbda), frameCount), np.dtype('float64'))

stop = False
threshold = 5.0

count = 0

20399 12239 4080 4080
12239 12239 4080 4080 4080 4080


In [5]:
# Uses bufTemp, y, m, alpha, lmbda, alpLmbTheta, loopCount, costArr
def learn():
    for idx, alp in enumerate(alpha):
        for idx2, lmbd in enumerate(lmbda):
            print(idx+1, idx2+1, alp, lmbd)
            for i in range(loopCount):
                cost = 0
                for j in range(trainM):
                    z = np.array(trainX[j], np.dtype('float64'))
#                     print(type(z), type(z[0]))
                    grad = ((np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j])*z)
#                     print(type(np.dot(z, alpLmbTheta[idx, idx2])))
#                     print(type(np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j]))
#                     print(type(((np.dot(z, alpLmbTheta[idx, idx2]) - trainY[j])*z)[0]))
#                     print(type(grad), type(grad[0]))
                    grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
                    alpLmbTheta[idx,idx2] = (alpLmbTheta[idx,idx2] - (alp*grad))
                    global count
                    count = count+1
                if i % costOnLoop == 0:
                    cost = np.zeros((1), np.dtype('float64'))
                    for j in range(1,frameCount):
                        k = j-1
#                         firstFrame = np.array(bufTemp[j-1], dtype='int16')
#                         secondFrame = np.array(bufTemp[j], dtype='int16')
#                         x = np.empty(((frameHeight*frameWidth)+1), np.dtype('float64'))
#                         x[0] = 1
#                         x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
#                         x = np.absolute(x)
#                         mean = np.mean(x)
#                         x = x - mean
#                         x = x/np.std(x)
                        result[idx][idx2][k] = np.dot(x[k],alpLmbTheta[idx,idx2])
#                         cost = cost + (((result[idx][idx2][j-1] - y[j])**2)/2)
                        cost = cost + (((result[idx][idx2][k] - y[j])**2))
#                     cost = cost/(2*m)
                    cost = cost/(m)
#                             global costArr, beginArr
#                             costArr = np.append(costArr, cost)
                    print("%d:"%(i), cost)

#                     if i >= 3:
#                         if (costArr[-2] - costArr[-1]) <  ((costArr[beginArr] - costArr[beginArr+1])/difDivider):
#                             alpha[idx] = alp/2
# #                             costArr = np.append(costArr, 0)
#                             beginArr = len(costArr)
#                             return 1

        print()

In [6]:
def startLearn():
    learn()
    while costArr[-1] > threshold:
        learn()

In [7]:
def startLearnRange():
    for i in range(outerLoopCount):
        alpha[0] = alpha2[(outerLoopCount-1)-i]
        learn()
#         print()

In [8]:
def getResult(X, setM):
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(setM):
                z = np.array(X[j], np.dtype('float64'))
                result[idx][idx2][j] = np.dot(z,alpLmbTheta[idx,idx2])

#     result.tofile('./train/trainMini.txt',sep="\n",format="%s")

In [9]:
def getCost(Y, setM):
    cost = np.zeros((1), np.dtype('float64'))
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(setM):
#                 cost = cost + (((result[idx][idx2][j] - y[j])**2)/2)
                cost = cost + ((result[idx][idx2][j] - Y[j])**2)
#             cost = cost/(2*m)
            cost = cost/(m)
            print(lmbda[idx2], ":", cost)
#             alpLmbCost[idx,idx2] = cost
#             print(alpha[idx], ":", cost)
        print()

In [ ]:
learn()

In [10]:
lmbda[0] = 0.64
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

1 1 0.1 0.64
0: [nan]


KeyboardInterrupt: 

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 1.28
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 2.56
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
lmbda[0] = 5.12
startLearnRange()
getResult(trainX, trainM)
print("Train:")
getCost(trainY, trainM)
getResult(valX, valM)
print("Val:")
getCost(valY, valM)
getResult(testX, testM)
print("Test:")
getCost(testY, testM)

In [ ]:
alpLmbTheta[0][0] = np.loadtxt('./train/trainThetaMini2.txt', np.dtype('float64'), delimiter='\n')
getResult()
getCost()

In [ ]:
alpLmbTheta.tofile('./train/trainThetaMini3.txt',sep="\n",format="%s")

In [ ]:
result.tofile('./train/trainMini.txt',sep="\n",format="%s")

In [ ]:
result = np.empty((len(alpha), len(lmbda), frameCount+2), np.dtype('float64'))
result[0][0] = np.loadtxt('./train/trainMini.txt', np.dtype('float64'), delimiter='\n')
getCost()